In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
import sklearn.feature_extraction
from sklearn.preprocessing import OneHotEncoder
from category_encoders import *
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn import linear_model
from sklearn import preprocessing

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
train.head()

In [ ]:
test.head()

In [2]:
train_y = train['Survived'].values
train.drop(['Survived'], inplace=True, axis=1)
#difference between test and train
[column for column in train.columns if column not in test.columns]


[]

# Feature Engineering

In [3]:
#extracting salutation as a feature
train['salutation'] = train['Name'].map(lambda x: x.split(' ')[1] )
#length of ticket ID
train['ticket_len'] = train['Ticket'].map(len)
#family size
train['fam_size'] = train['SibSp'] + train['Parch'] + 1
#treating missing values
train.fillna(-999, inplace=True)

In [4]:
categorical_columns = ['Sex','Cabin','Embarked','salutation']

In [5]:
#adding features to test data
test['salutation'] = test['Name'].map(lambda x: x.split(' ')[1] )
#length of ticket ID for each passenger
test['ticket_len'] = test['Ticket'].map(len)
#family size
test['fam_size'] = test['SibSp'] + test['Parch'] + 1

test.fillna(-999, inplace=True)

myEncoder = OneHotEncoder( handle_unknown='ignore')
myEncoder.fit(train[categorical_columns])

train_X = pd.concat([train.drop(categorical_columns+['Name','Ticket'], 1),
          pd.DataFrame(myEncoder.transform(train[categorical_columns]))], axis=1).reindex()
test_X = pd.concat([test.drop(categorical_columns+['Name','Ticket'], 1),
          pd.DataFrame(myEncoder.transform(test[categorical_columns]))], axis=1).reindex()

## Preliminary Linear Model

In [ ]:

logreg = linear_model.LogisticRegression(C=10000)
logreg.fit(train_X, train_y)

In [ ]:
y_pred = logreg.predict(train_X)
print(classification_report(train_y, y_pred))

In [ ]:
y_pred = logreg.predict(test_X)
pd.DataFrame({'Survived':y_pred}, index=test['PassengerId']).to_csv('sub.csv')

## Random Forest Model

In [6]:
#accuracy of .79904 using criterion='gini',max_depth=9,max_features= 'auto',n_estimators= 100
rf = RandomForestClassifier(criterion='gini',max_depth=9,max_features= 'auto',n_estimators= 100,  random_state=0)
rf.fit(train_X, train_y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=9, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [7]:
print(classification_report(train_y, rf.predict(train_X)))

             precision    recall  f1-score   support

          0       0.87      0.93      0.90       549
          1       0.88      0.78      0.83       342

avg / total       0.87      0.87      0.87       891



In [8]:

pd.DataFrame({'Survived':rf.predict(test_X)}, index=test['PassengerId']).to_csv('sub_tree_lab_encoded_10.csv')